In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ibm_watson_machine_learning import APIClient
import pandas as pd

Create a client instance

In [3]:
url = "https://us-south.ml.cloud.ibm.com"
apikey = "X"


wml_credentials = {
      "apikey": apikey,
      "url": url
}

client = APIClient(wml_credentials)
client.version

'1.0.308'

# Prepare your model archive

In [6]:
try:
    !rmdir model
except:
    True
     
try:
    !rm model.py
except:
    True
try:    
    !rm -r model  
except:
    True
try:    
    !rm model.tar.gz
except:
    True    


rmdir: failed to remove 'model': No such file or directory
rm: cannot remove 'model.py': No such file or directory
rm: cannot remove 'model': No such file or directory


In [7]:
     
!ls

In [8]:
%mkdir model

In [9]:
%%writefile model/main.py

from docplex.util.environment import get_environment
import pandas
from six import iteritems
from collections.abc import Mapping
from os.path import join, dirname, basename, splitext, exists
import glob

class _InputDict(dict):
    def __init__(self, directory, names):
        dict.__init__(self)
        self._directory = directory
        for k in names:
            dict.__setitem__(self, k, None)
        file='model_schema.json'
        if self._directory is not None:
            file  = "{0}/".format(self._directory) + file
        self.dtype_schemas = self.get_dtype_schemas( file)
    def __getitem__(self, key):
        if isinstance(key, str):
            item = dict.__getitem__(self, key)
            if item is None:
                file = "{0}.csv".format(key)
                if file in self.dtype_schemas:
                    return self.read_df( key, dtype=self.dtype_schemas[file])
                else:
                    return self.read_df( key)
            else:
                return item
        else:
            raise Exception("Accessing input dict via non string index")
    def read_df(self, key, **kwargs):
        env = get_environment()
        file = "{0}.csv".format(key)
        if self._directory is not None:
            file  = "{0}/".format(self._directory) + file
        with env.get_input_stream(file) as ist:
            params = {'encoding': 'utf8'}
            if kwargs:
                params.update(kwargs)
            df = pandas.read_csv( ist, **params)
            dict.__setitem__(self, key, df)
        return df
    def get_dtype_schemas(self, path):
        dtype_schemas = {}
        if exists(path):
            input_schemas=json.load(open(path))
            if 'input' in input_schemas:
                for input_schema in input_schemas['input']:
                    dtype_schema = {}
                    if 'fields' in input_schema:
                        for input_schema_field in input_schema['fields']:
                            if input_schema_field['type']=='string':
                                dtype_schema[input_schema_field['name']]='str'
                        if len(dtype_schema) > 0:
                            dtype_schemas[input_schema['id']]=dtype_schema
        print(dtype_schemas)
        return dtype_schemas

class _LazyDict(Mapping):
    def __init__(self, *args, **kw):
        self._raw_dict = _InputDict(*args, **kw)

    def __getitem__(self, key):
        return self._raw_dict.__getitem__(key)

    def __iter__(self):
        return iter(self._raw_dict)

    def __len__(self):
        return len(self._raw_dict)

    def read_df(self, key, **kwargs):
        return self._raw_dict.read_df(key, **kwargs)

def get_all_inputs(directory=None):
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''

    all_csv = "*.csv"
    g = join(directory, all_csv) if directory else all_csv

    names = [splitext(basename(f))[0] for f in glob.glob(g)]
    result = _LazyDict(directory, names)
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")

Writing model/main.py


In [10]:
%%writefile -a model/main.py

import pandas as pd
from docplex.mp.model import Model

def binary_var_series(df, mdl,**kargs):
    return pd.Series(mdl.binary_var_list(df.index, **kargs), index = df.index)



class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)


def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df   

inputs = get_all_inputs()

Nurse = inputs['Nurse']
Days = inputs['Days']
Shifts = inputs['Shifts']


MODEL_NAME = 'NurseScheduling'
SCENARIO_NAME = 'Base_Scenario' 

mdl = Model(MODEL_NAME)


Nurse_Shift = pd.merge(Days,Shifts, how = "cross" ).merge(Nurse, how = "cross").set_index(["DayID","ShiftsID","NurseID"], verify_integrity = True)
Nurse_Shift["X_Assigned"] = binary_var_series(Nurse_Shift, mdl, name = "X_Assigned" )
Nurse_Shift = Nurse_Shift.reset_index(drop = False)

Nurse_each_shift = Nurse_Shift[["DayID","ShiftsID","X_Assigned"]].groupby(["DayID","ShiftsID"]).agg(CplexSum(mdl)).reset_index(drop= False)


for row in Nurse_each_shift.itertuples():
    mdl.add_constraint(row.X_Assigned  == 1)

Nurse_each_day = Nurse_Shift[["NurseID","DayID","X_Assigned"]].groupby(["NurseID","DayID"]).agg(CplexSum(mdl)).reset_index(drop= False)

for row in Nurse_each_day.itertuples():
    mdl.add_constraint(row.X_Assigned  <= 1)

Nurse_day = Nurse_Shift[["NurseID","X_Assigned"]].groupby(["NurseID"]).agg(CplexSum(mdl)).reset_index(drop= False)

for row in Nurse_day.itertuples():
    mdl.add_constraint(row.X_Assigned  >= 2)

ok = mdl.solve(log_output=True)

if ok:
    mdl.print_solution()


    solution = extract_solution(Nurse_Shift, extract_dvar_names= ['X_Assigned'] ,drop=False)
    solution = solution[solution["X_Assigned_Solution"]>0.1]
    Schedule = solution.pivot(index = "DayID", columns = "ShiftsID", values = "NurseID")
    print(Schedule)
    outputs={}
    outputs['Schedule']=Schedule
    
# Generate output files
write_all_outputs(outputs)

Appending to model/main.py


In [11]:
!ls

model


In [12]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
#tar.add("model.py", arcname="model.py", filter=reset)
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

# Upload your model on Watson Machine Learning

In [13]:
# Find the space ID

space_name = 'Deployment_space_DO'

space_id = [x['metadata']['id'] for x in client.spaces.get_details()['resources'] if x['entity']['name'] == space_name][0]

client.set.default_space(space_id)

'SUCCESS'

In [14]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "MyModel",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Nurse Scheduling",
    client.repository.ModelMetaNames.TYPE: "do-docplex_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("do_22.1"),
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)
#model='/home/wsuser/work/model.tar.gz', 
model_uid = client.repository.get_model_id(model_details)

# Create a deployment

In [15]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Nurse Scheduling",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Nurse Scheduling",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S', 'num_nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '3f8e6c22-5908-4d07-b4ae-ded639c5f516' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3e355bf1-2a5e-435d-816d-5a9e300228f8'
------------------------------------------------------------------------------------------------


------------------------------------  --------------------  -----  ------------------------  -------------  ----------  ----------------
GUID                                  NAME                  STATE  CREATED                   ARTIFACT_TYPE  SPEC_STATE  SPEC_REPLACEMENT
3e355bf1-2a5e-435d-816d-5a9e300228f8  Nurse Scheduling      ready  2023-06-30T02:21:59.012Z  model          supported
3d6d1f12-4ef6-4dea-889a-c1a9b619f46f

,GUID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,3e355bf1-2a5e-435d-816d-5a9e300228f8,Nurse Scheduling,ready,2023-06-30T02:21:59.012Z,model,supported,
1,3d6d1f12-4ef6-4dea-889a-c1a9b619f46f,NurseModelDeployment,ready,2023-06-30T01:45:50.669Z,model,supported,


In [16]:
deployment_uid

'3e355bf1-2a5e-435d-816d-5a9e300228f8'

# Create another data and test with the model using a job

In [17]:
import pandas as pd 
  
Number_Nurses = 4
Number_Days = 3
Number_Shifts = 3

Nurse = pd.DataFrame([])
Nurse["NurseID"] = ["Nurse" +str(i) for i in range(1,Number_Nurses+1)]

Days = pd.DataFrame([])
Days["DayID"] = ["Day" +str(i) for i in range(1,Number_Days+1)]

Shifts = pd.DataFrame([])
Shifts["ShiftsID"] = ["Shifts" +str(i) for i in range(1,Number_Shifts+1)]

In [18]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [{
            "id":"Nurse.csv",
            "values" : Nurse
        },
        {
            "id":"Days.csv",
            "values" : Days
        },
        {
            "id":"Shifts.csv",
            "values" : Shifts
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)
job_uid

'46eb0ccb-0b0c-4e5c-a401-9da232732bd9'

In [19]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
running...
completed


In [20]:
job_details['entity']['decision_optimization']['output_data']

[{'fields': ['Shifts1', 'Shifts2', 'Shifts3'],
  'id': 'Schedule.csv',
  'values': [['Nurse3', 'Nurse1', 'Nurse4'],
   ['Nurse3', 'Nurse2', 'Nurse1'],
   ['Nurse2', 'Nurse3', 'Nurse4']]},
 {'fields': ['Name', 'Value'],
  'id': 'stats.csv',
  'values': [['cplex.modelType', 'MILP'],
   ['cplex.size.integerVariables', 0],
   ['cplex.size.continousVariables', 0],
   ['cplex.size.linearConstraints', 25],
   ['cplex.size.booleanVariables', 36],
   ['cplex.size.constraints', 25],
   ['cplex.size.quadraticConstraints', 0],
   ['cplex.size.variables', 36],
   ['job.coresCount', 1],
   ['job.inputsReadMs', 2161],
   ['job.memoryPeakKB', 100852],
   ['job.modelProcessingMs', 357]]}]

In [21]:
print(client.deployments.get_job_details(job_uid)['entity']['decision_optimization']['status'])

{'completed_at': '2023-06-30T02:22:28.662Z', 'running_at': '2023-06-30T02:22:25.925Z', 'state': 'completed'}


# delete the deployment

In [17]:
client.deployments.delete(deployment_uid)

'SUCCESS'

In [18]:
solve_payload

{'input_data': [{'id': 'Nurse.csv',
   'values': [['Nurse1'], ['Nurse2'], ['Nurse3'], ['Nurse4']],
   'fields': ['NurseID']},
  {'id': 'Days.csv',
   'values': [['Day1'], ['Day2'], ['Day3']],
   'fields': ['DayID']},
  {'id': 'Shifts.csv',
   'values': [['Shifts1'], ['Shifts2'], ['Shifts3']],
   'fields': ['ShiftsID']}],
 'output_data': [{'id': '.*\\.csv'}]}